# **Multi gpu - CHPS0904 - Multi GPU Programming Models for HPC and AI**

# Partie 1 : Version CPU 

## Fonctionnement global

- **Initialisation** : la grille 2D est initialisée avec des conditions aux limites (bords gauche et droit à 1, le reste à 0).
- **Itérations de Jacobi** : à chaque itération, chaque point intérieur de la grille est mis à jour en prenant la moyenne de ses 4 voisins.
- **Application des conditions périodiques** : les lignes du haut et du bas sont reliées (périodicité en Y).
- **Critère d’arrêt** : l’algorithme s’arrête si l’erreur maximale entre deux itérations consécutives est inférieure à la tolérance demandée, ou si le nombre maximal d’itérations est atteint.
- **Affichage et analyse** : le programme affiche l’état final de la grille, les valeurs aux coins, au centre, et le temps de calcul.

---

## Explications code

### 1. Initialisation de la grille

```c
void initialize(double *a, int nx, int ny, double left, double right) {
    for (int iy = 0; iy < ny; iy++) {
        int base = iy * nx;
        a[base] = left; // bord gauche
        a[base + nx - 1] = right; // bord droit
        for (int ix = 1; ix < nx - 1; ix++) {
            a[base + ix] = 0.0;
        }
    }
}
```
Cette fonction initialise chaque ligne de la grille :
- La première et la dernière colonne (bords gauche et droit) sont mises à la valeur choisie (ici 1.0).
- Toutes les autres cases (l’intérieur de la grille) sont initialisées à 0.0.
- Cette étape est appelée pour les deux tableaux `a` et `a_new` avant d’entrer dans la boucle principale.

---

### 2. Étape de Jacobi : mise à jour des points intérieurs

```c
void jacobi_step(double *a_new, double *a, int nx, int ny) {
    for (int iy = 1; iy < ny - 1; iy++) {
        int row = iy * nx;
        int row_up = (iy - 1) * nx;
        int row_down = (iy + 1) * nx;
        for (int ix = 1; ix < nx - 1; ix++) {
            a_new[row + ix] = 0.25 * (
                a[row_up + ix] +     // voisin du haut
                a[row_down + ix] +   // voisin du bas
                a[row + (ix - 1)] +  // voisin de gauche
                a[row + (ix + 1)]    // voisin de droite
            );
        }
    }
}
```
- Cette fonction met à jour chaque point intérieur de la grille (on ne touche pas aux bords).
- Pour chaque point, on prend la moyenne de ses quatre voisins immédiats (haut, bas, gauche, droite) de l’ancienne itération et on stocke le résultat dans `a_new`.
- L’utilisation de variables `row`, `row_up`, `row_down` permet de rendre le code plus lisible et efficace.

---

### 3. Application des conditions périodiques en Y

```c
void apply_periodic_bc(double *a, int nx, int ny) {
    int base_last = (ny - 2) * nx;
    int base_first = 1 * nx;
    // Ligne du haut
    for (int ix = 0; ix < nx; ix++) {
        a[ix] = a[base_last + ix];
    }
    // Ligne du bas
    int base_bottom = (ny - 1) * nx;
    for (int ix = 0; ix < nx; ix++) {
        a[base_bottom + ix] = a[base_first + ix];
    }
}
```
- Cette fonction copie la première ligne intérieure sur la ligne du bas et la dernière ligne intérieure sur la ligne du haut, imposant ainsi une périodicité en Y.
- Cela signifie que le haut et le bas de la grille sont connectés.

---

### 4. Boucle principale et calcul de l’erreur

Dans la fonction `main`, la boucle principale s’occupe de :
- Lancer une étape de Jacobi,
- Appliquer les conditions périodiques,
- Calculer l’erreur maximale sur tous les points intérieurs :

```c
error = 0.0;
for (iy = 1; iy < ny - 1; iy++) {
    for (ix = 1; ix < nx - 1; ix++) {
        int tot = iy*nx;
        double diff = fabs(a_new[tot + ix] - a[tot + ix]);
        if (diff > error) error = diff;
    }
}
```

- Échanger les pointeurs des tableaux (pour ne pas recopier les valeurs) :

```c
double *tmp = a;
a = a_new;
a_new = tmp;
```

- Incrémenter le compteur d’itération.

---

### 5. Critère d’arrêt et affichage

- La boucle s’arrête si `error <= tol` ou si `iter >= max_iter`.
- À la fin, on affiche :
    - Le nombre d’itérations,
    - L’erreur finale et l’état de la convergence,
    - Le temps total écoulé,
    - Les valeurs aux coins et au centre de la grille.

---

## Compilation 

In [59]:
%%bash
cd CPU
make clean 
make


rm -f jacobi_seq
gcc -O3 -Wall -march=native -funroll-loops -ffast-math -std=c99 -fopenmp -o jacobi_seq jacobi.c laplace2d.c -lm


## Execution

In [36]:
%%bash 
./CPU/jacobi_seq 4096 4096 1000 1e-6 0

Jacobi solver stopped after 1000 iterations
Reached max_iter = 1000 (error = 2.419263e-04 > tol = 1.000000e-06)
Elapsed time   : 15.464313 seconds
Values at corners: [0,0]=1.00  [0,4095]=1.00
Value at center [2048,2048] = 0.000000


# Partie 2 : Version CPU + GPU 


## Fonctionnement global

- **Initialisation** : la grille 2D est initialisée côté host (CPU) puis copiée sur le device (GPU).
- **Itérations de Jacobi** : à chaque itération, un kernel CUDA met à jour tous les points intérieurs de la grille sur le GPU.
- **Application des conditions périodiques** : un kernel CUDA copie les bonnes lignes pour gérer la périodicité en Y.
- **Critère d’arrêt** : on utilise Thrust pour calculer, sur le GPU, l’erreur maximale entre deux itérations consécutives, sans transfert sur le CPU.
- **Affichage et analyse** : le programme copie le résultat sur le CPU pour affichage et donne le temps de calcul.

---

## Explications code

### 1. Initialisation sur le CPU et transfert sur le GPU

```c
void initialize_host(double* a, int nx, int ny, double left, double right) {
    for (int iy = 0; iy < ny; iy++) {
        int base = iy * nx;
        a[base] = left;
        a[base + nx - 1] = right;
        for (int ix = 1; ix < nx-1; ix++) {
            a[base + ix] = 0.0;
        }
    }
}
```
- Cette fonction initialise la grille sur le CPU : bords gauche/droit à 1, reste à 0.
- Ensuite, on copie ces données sur la mémoire GPU avec `cudaMemcpy`.

---

### 2. Étape de Jacobi : kernel CUDA pour la mise à jour

```c
__global__ void jacobi_step(double* a_new, const double* a, int nx, int ny) {
    int ix = blockIdx.x * blockDim.x + threadIdx.x;
    int iy = blockIdx.y * blockDim.y + threadIdx.y;
    if (ix > 0 && ix < nx-1 && iy > 0 && iy < ny-1) {
        int idx = iy * nx + ix;
        a_new[idx] = 0.25 * (
            a[idx - nx] +   // voisin du haut
            a[idx + nx] +   // voisin du bas
            a[idx - 1 ] +   // voisin de gauche
            a[idx + 1 ]     // voisin de droite
        );
    }
}
```
- Ce kernel exécute la mise à jour Jacobi sur **tous les points intérieurs** de la grille en parallèle sur le GPU.
- Chaque thread traite un point `(ix, iy)`.

---

### 3. Application des conditions périodiques en Y sur GPU

```c
__global__ void apply_periodic_bc(double* a, int nx, int ny) {
    int ix = blockIdx.x * blockDim.x + threadIdx.x;
    if (ix < nx) {
        int top    = 0 * nx + ix;
        int bottom = (ny-1) * nx + ix;
        int first  = 1 * nx + ix;
        int last   = (ny-2) * nx + ix;
        a[top]    = a[last];
        a[bottom] = a[first];
    }
}
```
- Ce kernel copie la première ligne intérieure sur la ligne du bas, et la dernière ligne intérieure sur la ligne du haut pour respecter la périodicité.

---

### 4. Calcul de l’erreur de convergence avec Thrust

```c
struct max_abs_diff {
    __host__ __device__
    double operator()(const thrust::tuple<double,double>& t) const {
        return fabs(thrust::get<0>(t) - thrust::get<1>(t));
    }
};
...
thrust::device_ptr<double> ptr_new(d_a_new);
thrust::device_ptr<double> ptr_old(d_a);
error = thrust::transform_reduce(
    thrust::make_zip_iterator(thrust::make_tuple(ptr_new, ptr_old)),
    thrust::make_zip_iterator(thrust::make_tuple(ptr_new + N, ptr_old + N)),
    max_abs_diff(),
    0.0,
    thrust::maximum<double>());
```
Thrust est une bibliothèque C++ intégrée à CUDA qui fournit des primitives algorithmiques (comme les réductions, tris, scans) optimisées pour le GPU. Ici, j'utilise `thrust::transform_reduce` pour parcourir les tableaux sur le GPU et calculer en parallèle l’erreur maximale entre deux itérations, sans repasser les données sur le CPU.
- transform : applique le foncteur max_abs_diff à chaque paire (a_new, a_old) → calcule la différence absolue pour chaque point.
- reduce : trouve le maximum de toutes ces différences (critère de convergence Jacobi)Tout ça SANS aller-retour entre GPU et CPU !
      
- Le **foncteur** `max_abs_diff` calcule la différence absolue entre chaque case des deux grilles (état courant et précédent).
- `thrust::transform_reduce` applique ce foncteur à tous les éléments du tableau, puis prend le maximum : on obtient ainsi l’erreur max **directement sur le GPU**, sans rapatrier les données.

---

### 5. Organisation de la boucle principale

Dans `main`, la boucle effectue :
- Une étape Jacobi sur le GPU (`jacobi_step`)
- L’application des conditions périodiques (`apply_periodic_bc`)
- Le calcul de l’erreur de convergence avec Thrust
- L’échange des pointeurs device (pour éviter des copies de tableaux)
- L’incrémentation du compteur d’itérations

---

### 6. Gestion des temps

- on utilise `cudaEvent` pour mesurer le temps de calcul CUDA uniquement, et `clock()` pour mesurer le temps global (CPU + GPU, initialisation comprise).

---

### 7. Récupération et affichage des résultats

- À la fin, le résultat final est copié du GPU vers le CPU pour affichage.
- Affichage du nombre d’itérations, erreur finale, temps CUDA, et temps total du programme.

---


## Compilation 

In [11]:
%%bash
cd CPU-GPU
make

nvcc -O3 -arch=sm_60 -std=c++11        -o jacobi_cuda jacobi.cu -lm                             


## Execution

In [15]:
%%bash 
./CPU-GPU/jacobi_cuda 4096 4096 1000 1e-6 0

Solveur Jacobi CUDA convergé en 1000 itérations (erreur = 2.419e-04)
Temps passé (calcul CUDA) : 0.239045 s
Temps total du programme (tout inclus) : 2.320342 s


# Partie 3 : MPI + GPUs



## 1. Introduction de MPI

- Ajout de `#include <mpi.h>`.
- Initialisation de MPI en début de programme (`MPI_Init`) et terminaison en fin (`MPI_Finalize`).
- Chaque processus MPI correspond à un sous-domaine de la grille et, typiquement, à un GPU distinct (sélectionné automatiquement avec `cudaSetDevice(rank % num_gpus)`).

---

## 2. Découpage de la grille

- La grille globale est **découpée horizontalement** en sous-domaines : chaque processus MPI gère un bloc de lignes consécutives (appelées localement `local_ny`), plus deux lignes "halo" (fantômes) pour les échanges avec les voisins.
- Calcul de la répartition : gestion du cas où le nombre de lignes n’est pas un multiple exact du nombre de processus.
- Pour chaque processus :
    - `local_ny` : nombre de lignes propres à traiter
    - `ghost_ny = local_ny + 2` : nombre total de lignes allouées localement (incluant halos haut et bas)

---

## 3. Initialisation et mémoire

- Chaque processus alloue **sa portion locale** sur le GPU (`cudaMalloc`).
- Initialisation des bords effectuée sur cette portion locale (kernel `init_boundaries`).

---

## 4. Échanges des halos (communication MPI)

- **À chaque itération, chaque processus échange ses lignes de halo** avec ses voisins direct (haut et bas) via `MPI_Sendrecv` :
    - Envoie sa première ligne intérieure à son voisin du haut et reçoit le halo du haut.
    - Envoie sa dernière ligne intérieure à son voisin du bas et reçoit le halo du bas.
- Ces échanges se font directement entre les buffers device (`d_a + IDX(...)`)

---

## 5. Adaptation des kernels

- Les kernels CUDA (`jacobi_kernel` et `init_boundaries`) opèrent sur la portion locale avec gestion des halos.
- Les indices sont adaptés pour ne jamais toucher les bords globaux (gérés séparément si besoin).

---

## 6. Réduction de l’erreur et convergence globale

- Chaque processus calcule **l’erreur maximale locale** via Thrust.
- La convergence globale est obtenue en combinant toutes les erreurs locales par une réduction MPI :
    - `MPI_Allreduce` avec l’opération `MPI_MAX` pour obtenir l’erreur maximale sur tous les processus.

---

## 7. Synchronisation et chronométrage

- Synchronisation avec `MPI_Barrier` avant et après la boucle principale pour bien mesurer le temps de calcul pur.
- Deux chronos affichés : temps scientifique (calcul seul) et temps total du programme.

---

## 8. Rassemblement et affichage des résultats

- À la fin, chaque processus envoie son sous-domaine au processus 0 via `MPI_Send` / `MPI_Recv` pour permettre un affichage complet.
- Reconstruction de la grille globale sur le processus 0, avec gestion de la périodicité sur les bords pour l’affichage.

---

## 9. Libération des ressources

- Libération de la mémoire GPU pour chaque processus.
- Nettoyage MPI via `MPI_Finalize`.

---

## Compilation

In [3]:
%%bash
cd MPI-GPU
make nsight



mkdir -p nsight
# Adapte la ligne ci-dessous selon les arguments à donner à ton programme
nsys profile -o nsight/jacobi_cuda_profile \
	mpirun -np 4 ./jacobi_cuda 4096 4096 10000 1e-6 0
Solveur Jacobi MPI+multiGPU convergé en 10000 itérations (erreur = 2.420e-05)
Temps calcul MPI+multiGPU (hors init): 10.474947 s
Temps total du programme (allocs, MPI, init, calcul, etc.): 12.065351 s
Generating '/tmp/nsys-report-35d9.qdstrm'
[1/1] [========================100%] jacobi_cuda_profile.nsys-rep
Generated:
    /gpfs/home/scortinhal/CHPS0904/MultiGPU/MPI-GPU/nsight/jacobi_cuda_profile.nsys-rep


In [37]:
%%bash
cd MPI-GPU

mpirun -np 4 ./jacobi_cuda 4096 4096 1000 1e-6 0



Solveur Jacobi MPI+multiGPU convergé en 1000 itérations (erreur = 2.422e-04)
Temps calcul MPI+multiGPU (hors init): 1.188106 s
Temps total du programme (allocs, MPI, init, calcul, etc.): 1.944380 s


![MPI-GPU](MPI-GPU/nsight/MPI-GPU.PNG)

# Partie 4 : MPI + Overlap


---

## 1. Ajout de CUDA streams multiples

Pour permettre d’exécuter en parallèle transferts mémoire et calculs sur différentes parties de la grille, plusieurs streams CUDA sont déclarés et créés :
```c
cudaStream_t stream_halo_top, stream_halo_bot, stream_interior, stream_memcpy;
cudaStreamCreate(&stream_halo_top);
cudaStreamCreate(&stream_halo_bot);
cudaStreamCreate(&stream_interior);
cudaStreamCreate(&stream_memcpy);
```

---

## 2. Buffers de halos côté host

Des tampons explicites pour l’envoi/réception des halos sont alloués :
```c
double *h_halo_top_send = (double*)malloc(nx * sizeof(double));
double *h_halo_top_recv = (double*)malloc(nx * sizeof(double));
double *h_halo_bot_send = (double*)malloc(nx * sizeof(double));
double *h_halo_bot_recv = (double*)malloc(nx * sizeof(double));
```
Ces buffers servent pour les transferts mémoire asynchrones et les échanges MPI non bloquants.

---

## 3. Boucle principale : pipeline overlap calcul/transferts/comm

### a. Copies device->host asynchrones des halos
```c
cudaMemcpyAsync(h_halo_top_send, d_a + IDX(1,0,nx), nx*sizeof(double), cudaMemcpyDeviceToHost, stream_memcpy);
cudaMemcpyAsync(h_halo_bot_send, d_a + IDX(local_ny,0,nx), nx*sizeof(double), cudaMemcpyDeviceToHost, stream_memcpy);
cudaStreamSynchronize(stream_memcpy);
```
On lance la copie sur un stream dédié et on synchronise juste ce stream, sans bloquer le calcul sur le reste de la grille.

### b. Communications MPI non bloquantes pour les halos
```c
MPI_Request reqs[4];
MPI_Irecv(h_halo_top_recv, nx, MPI_DOUBLE, prev, 1, MPI_COMM_WORLD, &reqs[0]);
MPI_Irecv(h_halo_bot_recv, nx, MPI_DOUBLE, next, 0, MPI_COMM_WORLD, &reqs[1]);
MPI_Isend(h_halo_top_send, nx, MPI_DOUBLE, prev, 0, MPI_COMM_WORLD, &reqs[2]);
MPI_Isend(h_halo_bot_send, nx, MPI_DOUBLE, next, 1, MPI_COMM_WORLD, &reqs[3]);
```
Cela permet de continuer les calculs sans attendre la fin des communications.

### c. Calcul du cœur de la grille en overlap pendant le MPI
```c
int iy_start_interior = 2;
int iy_end_interior = local_ny;
if (iy_end_interior > iy_start_interior) {
    dim3 grid_interior((nx+BLOCK-1)/BLOCK, (iy_end_interior-iy_start_interior+BLOCK-1)/BLOCK);
    jacobi_kernel_slice<<<grid_interior, block, 0, stream_interior>>>(d_a, d_a_new, nx, iy_start_interior, iy_end_interior);
}
```
On lance immédiatement le calcul Jacobi sur la partie intérieur (lignes intérieures sans dépendance aux halos).

### d. Synchronisation minimale, gestion des halos reçus
```c
MPI_Wait(&reqs[0], MPI_STATUS_IGNORE); // halo du haut
MPI_Wait(&reqs[1], MPI_STATUS_IGNORE); // halo du bas

cudaMemcpyAsync(d_a + IDX(0,0,nx), h_halo_top_recv, nx*sizeof(double), cudaMemcpyHostToDevice, stream_memcpy);
cudaMemcpyAsync(d_a + IDX(ghost_ny-1,0,nx), h_halo_bot_recv, nx*sizeof(double), cudaMemcpyHostToDevice, stream_memcpy);
cudaStreamSynchronize(stream_memcpy);
```
On ne synchronise que ce qui est nécessaire, afin de lancer le calcul sur les bandes extrêmes dès que possible.

### e. Calcul des bandes extrêmes (haut et bas)
```c
dim3 grid_band((nx+BLOCK-1)/BLOCK, 1);
jacobi_kernel_slice<<<grid_band, block, 0, stream_halo_top>>>(d_a, d_a_new, nx, 1, 2);
jacobi_kernel_slice<<<grid_band, block, 0, stream_halo_bot>>>(d_a, d_a_new, nx, local_ny, local_ny+1);
```
On traite les lignes qui dépendent des halos dans leurs propres streams, après leur arrivée.

### f. Attentes sélectives des streams CUDA
```c
cudaStreamSynchronize(stream_interior);
cudaStreamSynchronize(stream_halo_top);
cudaStreamSynchronize(stream_halo_bot);
```
On attend la fin des calculs avant la réduction.

### g. Attente de la fin des communications MPI
```c
MPI_Waitall(4, reqs, MPI_STATUSES_IGNORE);
```

---

## 4. Nettoyage spécifique

À la fin du programme, il faut maintenant libérer explicitement les nouveaux objets alloués :
```c
cudaStreamDestroy(stream_halo_top);
cudaStreamDestroy(stream_halo_bot);
cudaStreamDestroy(stream_interior);
cudaStreamDestroy(stream_memcpy);

free(h_halo_top_send); free(h_halo_top_recv);
free(h_halo_bot_send); free(h_halo_bot_recv);
```

---

## 5. Résumé des apports principaux

- **Overlap calcul/communication** : Superposition des calculs et des échanges MPI grâce aux copies asynchrones, aux streams CUDA multiples et aux MPI non bloquants.
- **Découpage fin** : Séparation explicite des calculs sur l’intérieur et sur les bandes extrêmes de la grille.
- **Gestion des dépendances** : Synchronisations minimales et localisées pour maximiser l’occupation GPU.

---

## 6. Remarque sur les kernels

Un nouveau kernel `jacobi_kernel_slice` est introduit :
```c
__global__
void jacobi_kernel_slice(const double *a, double *a_new, int nx, int iy_start, int iy_end) { ... }
```
Il permet de ne traiter qu’une **bande verticale** de la grille, paramétrable par indices, indispensable pour l’overlap.
---

## Compilation et execution


In [38]:
%%bash
cd MPI-GPU-overlap
make
mpirun -np 4 ./jacobi_cuda 4096 4096 1000 1e-6 0

make: Nothing to be done for 'all'.
Solveur Jacobi MPI+multiGPU (full overlap memcpyasync) convergé en 1000 itérations (error = 2.422e-04)
Temps calcul MPI+multiGPU (hors alloc/init): 0.937806 s
Temps total du programme (tout inclus): 1.822555 s


# Partie 5 : NCCL

## NCCL ?

- **NCCL** (NVIDIA Collective Communications Library) permet de faire des transferts **directement GPU-to-GPU** (device-to-device) même sur plusieurs machines équipées de NVLink ou d’Infiniband.
- Cela évite les copies host/device et la synchronisation CPU, ce qui réduit considérablement la latence d’échange entre GPU et augmente l’overlap.
- **NCCL est asynchrone** et s’intègre naturellement avec les streams CUDA, donc le calcul, les transferts et la communication peuvent être superposés sans blocage.
- Il est utilisé ici pour échanger les halos (lignes fantômes) entre sous-domaines de la grille Jacobi, mais peut également servir à la réduction globale (AllReduce, etc.).

---

## 1. Initialisation de NCCL dans MPI

Après avoir choisi le GPU local :
```c
int num_gpus = 0;
cudaGetDeviceCount(&num_gpus);
cudaSetDevice(rank % num_gpus);
```
**NCCL doit être initialisé et chaque rang reçoit le même identifiant :**
```c
ncclUniqueId id;
if (rank == 0) ncclGetUniqueId(&id);
MPI_Bcast(&id, sizeof(id), MPI_BYTE, 0, MPI_COMM_WORLD);
ncclComm_t nccl_comm;
ncclCommInitRank(&nccl_comm, size, id, rank);
```
- `ncclGetUniqueId(&id);`  
  (rendu par le rank 0) crée un identifiant de communicateur unique pour NCCL.
- `MPI_Bcast(...)`  
  Le communique à tous les processus via MPI.
- `ncclCommInitRank(...)`  
  Crée un contexte NCCL sur chaque rang avec ce communicateur, le nombre total de rangs, et le rang courant.

---

## 2. Utilisation de NCCL pour l’échange des halos

Au lieu des traditionnels `MPI_Sendrecv`, on utilise :
```c
cudaStream_t stream;
cudaStreamCreate(&stream);

while (error > tol && iter < max_iter) {
    // Echange des halos avec NCCL (overlap possible)
    ncclGroupStart();
    ncclRecv(d_a + IDX(0,0,nx), nx, ncclDouble, prev, nccl_comm, stream);
    ncclSend(d_a + IDX(1,0,nx), nx, ncclDouble, prev, nccl_comm, stream);
    ncclRecv(d_a + IDX(ghost_ny-1,0,nx), nx, ncclDouble, next, nccl_comm, stream);
    ncclSend(d_a + IDX(local_ny,0,nx), nx, ncclDouble, next, nccl_comm, stream);
    ncclGroupEnd();
    cudaStreamSynchronize(stream);

    // ...
}
```
**Explication de chaque commande NCCL :**
- `ncclGroupStart(); ... ncclGroupEnd();`  
  Grouper plusieurs opérations NCCL permet d’optimiser la communication et de lancer tous les échanges de façon atomique.
  ncclGroupStart() indique le début d'un groupe d'opérations de communication NCCL.
  ncclGroupEnd() marque la fin du groupe.
- `ncclRecv(...)`  
  Reçoit une ligne halo depuis le GPU voisin, directement en mémoire device, sur le stream CUDA donné.
- `ncclSend(...)`  
  Envoie la ligne halo correspondante à ce voisin, toujours device-to-device.
- Tous les échanges sont lancés en overlap sur le même stream.
- `cudaStreamSynchronize(stream);`  
  S’assure que tous les transferts sont finis avant de passer au calcul.

---

## 3. Boucle principale : pipeline calcul/communication

- Après les échanges NCCL, on lance le calcul Jacobi comme avant :
```c
jacobi_kernel<<<grid,block,0,stream>>>(d_a, d_a_new, nx, ghost_ny);
cudaStreamSynchronize(stream);
```
- On fait le swap des pointeurs, puis on utilise Thrust sur le GPU pour l’erreur locale, puis une `MPI_Allreduce` pour la convergence globale.

---

## 4. Nettoyage spécifique

En fin de programme, il faut maintenant :
```c
cudaStreamDestroy(stream);
ncclCommDestroy(nccl_comm);
```
pour libérer le communicateur NCCL et le stream CUDA.

---

## 5. Comparaison avec la version MPI+CUDA classique

- **Communication device-to-device** au lieu de device→host→MPI→host→device.
- **Overlapping naturel** grâce à l’intégration CUDA streams/NCCL.
- **Suppression des tampons host pour les halos** : tout est fait sur le device.
- Initialisation NCCL requiert une étape supplémentaire (diffusion d’un identifiant entre ranks).
- Tout le reste du code (calcul, Thrust, swap, affichage) reste quasi identique.

---




## Compilation et execution

In [21]:
%%bash
cd NCCL

eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH

make
mpirun -np 4 ./jacobi_nccl 4096 4096 1000 1e-6 0


make: Nothing to be done for 'all'.
Solveur Jacobi NCCL convergé en 1000 itérations (error = 2.422e-04)
1. Temps total du programme (MPI_Init → fin)                : 6.308566 s
2. Temps après init NCCL (juste avant alloc/init CUDA)      : 1.565614 s
3. Temps NCCL (calcul pur boucle Jacobi)                    : 1.562996 s


# Partie 6 : NCCL - Overlap


## 1. Initialisation du communicateur NCCL

On initialise NCCL comme dans la version simple :
```c
ncclUniqueId id;
if (rank == 0) ncclGetUniqueId(&id);
MPI_Bcast(&id, sizeof(id), MPI_BYTE, 0, MPI_COMM_WORLD);
ncclComm_t nccl_comm;
ncclCommInitRank(&nccl_comm, size, id, rank);
```
- Permet à chaque rang de communiquer directement en device-to-device.

---

## 2. **Streams CUDA multiples** pour overlap

Au lieu d'un seul stream pour tout, on crée :
```c
cudaStream_t stream_halo_top, stream_halo_bot, stream_interior;
cudaStreamCreate(&stream_halo_top);
cudaStreamCreate(&stream_halo_bot);
cudaStreamCreate(&stream_interior);
```
- **Un stream par halo** (haut/bas) pour la communication.
- **Un stream pour l'intérieur** pour le calcul bulk.

---

## 3. **Échange des halos device-to-device avec NCCL, en overlap**

On utilise **NCCL group** pour regrouper toutes les comms et on attribue **chaque halo à son stream** :
```c
ncclGroupStart();
ncclRecv(d_a + IDX(0,0,nx), nx, ncclDouble, prev, nccl_comm, stream_halo_top); // halo haut
ncclSend(d_a + IDX(1,0,nx), nx, ncclDouble, prev, nccl_comm, stream_halo_top);
ncclRecv(d_a + IDX(ghost_ny-1,0,nx), nx, ncclDouble, next, nccl_comm, stream_halo_bot); // halo bas
ncclSend(d_a + IDX(local_ny,0,nx), nx, ncclDouble, next, nccl_comm, stream_halo_bot);
ncclGroupEnd();
```
- **`ncclRecv`** : reçoit la ligne fantôme (halo) du voisin (top ou bottom), directement dans le device buffer, sur le stream dédié.
- **`ncclSend`** : envoie la ligne fantôme calculée au voisin, device-to-device, sur le même stream.
- L’usage d’un stream par direction permet d’overlapper communication et calcul.

---

## 4. **Calcul en overlap sur plusieurs streams**

Après avoir lancé les communications, on peut immédiatement lancer le calcul sur les bandes et l'intérieur :
```c
// Calcul extrémités (dépendants des halos) chacun sur leur stream
dim3 grid_band((nx+BLOCK-1)/BLOCK, 1);
jacobi_kernel_slice<<<grid_band, block, 0, stream_halo_top>>>(d_a, d_a_new, nx, 1, 2);
jacobi_kernel_slice<<<grid_band, block, 0, stream_halo_bot>>>(d_a, d_a_new, nx, local_ny, local_ny+1);

// Calcul bulk (indépendant des halos) sur un troisième stream
int iy_start_interior = 2;
int iy_end_interior = local_ny;
if (iy_end_interior > iy_start_interior) {
    dim3 grid_interior((nx+BLOCK-1)/BLOCK, (iy_end_interior-iy_start_interior+BLOCK-1)/BLOCK);
    jacobi_kernel_slice<<<grid_interior, block, 0, stream_interior>>>(d_a, d_a_new, nx, iy_start_interior, iy_end_interior);
}
```
- Permet d’exécuter les calculs sur les régions qui n’attendent pas la fin des communications pendant que les halos arrivent.

---

## 5. **Synchronisation fine**

Pour garantir la cohérence, chaque stream est synchronisé individuellement avant la réduction :
```c
cudaStreamSynchronize(stream_halo_top);
cudaStreamSynchronize(stream_halo_bot);
cudaStreamSynchronize(stream_interior);
```

---

## 6. **Nettoyage**

Libération explicite des nouveaux streams :
```c
cudaStreamDestroy(stream_halo_top);
cudaStreamDestroy(stream_halo_bot);
cudaStreamDestroy(stream_interior);
ncclCommDestroy(nccl_comm);
```

---

## 7. **Résumé de l’intérêt**

- **Overlap optimal** communication/calcul : réduit le temps mort lié à la latence réseau.
- **Tout reste sur le device** : plus aucun transfert inutile device<->host pour les halos.
- **Kernels spécialisés** et lancement sur plusieurs streams permettent d’exploiter pleinement le GPU même si la comm est en attente.

---

## 8. **Points-clés sur NCCL**

- `ncclRecv/Send` : transferts directs entre GPUs (device-to-device) sur le stream spécifié.
- `ncclGroupStart/End` : regroupe toutes les ops de comms pour maximiser le pipelining.
- **Aucune barrière explicite MPI dans la boucle**, seul MPI_Allreduce pour la convergence.
- **Tout le découpage reste identique** (partition des lignes entre rangs).

---

## Compilation et execution


In [34]:
%%bash
cd NCCL-overlap

eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH
export NCCL_IB_HCA=mlx5_0         # si tu as Infiniband
export NCCL_IB_DISABLE=0
export NCCL_P2P_DISABLE=1
make
mpirun -np 4 ./jacobi_nccl 4096 4096 1000 1e-6 0


make: Nothing to be done for 'all'.
NCCL Comm Init: 2.270 s
Solveur Jacobi NCCL + overlap convergé en 1000 itérations (error = 2.422e-04)
1. Temps total du programme (MPI_Init → fin)                 : 5.631847 s
2. Temps après init NCCL (juste avant alloc/init CUDA)       : 0.912701 s
3. Temps calcul NCCL + overlap (boucle Jacobi uniquement)    : 0.909242 s


![MPI-GPU](NCCL/nsight/NCCL-nsight.PNG)

# Partie 7 : NCCL - graph

## Graph ? 
Les CUDA Graphs sont une fonctionnalité de CUDA qui permet de capturer (enregistrer) toute une séquence d’opérations (kernels, transferts mémoire, communications, etc.) dans un graphe orienté acyclique (DAG), puis de la lancer d’un seul coup sur le GPU.
Objectifs et avantages principaux

- Réduire l’overhead CPU :
    Quand tu lances des centaines/milliers de kernels dans une boucle, chaque appel CPU→GPU a un coût (latence, passage de paramètres…).
    Avec un graph, tu encapsules toute la séquence : le CPU ne fait qu’un seul appel par itération.

- Pipeline matériel :
    Le GPU peut analyser le graphe et planifier/exécuter les tâches de façon plus efficace, notamment en pipeline ou en parallélisant ce qui peut l’être.

- Optimisation automatique :
    CUDA peut mieux optimiser les transferts, le scheduling, et profiter des dépendances entre opérations.

- Mutualisation avec communication :
    Si tu inclues aussi des transferts NCCL/MPI dans le graph, tu maximises l’overlap calcul/comm sans avoir à tout gérer manuellement côté CPU.

## 1. **Gestion de l’affinité GPU / MPI (multi-nœud)**

Chaque rang MPI choisit son GPU en se basant sur `local_rank` pour éviter les conflits sur un nœud multi-GPU :

```c
int local_rank = 0;
{
    MPI_Comm local_comm;
    MPI_Comm_split_type(MPI_COMM_WORLD, MPI_COMM_TYPE_SHARED, rank, MPI_INFO_NULL, &local_comm);
    MPI_Comm_rank(local_comm, &local_rank);
    MPI_Comm_free(&local_comm);
}
cudaSetDevice(local_rank);
```
- **But :** pas de collisions entre plusieurs processus MPI du même nœud.

---

## 2. **Un seul stream CUDA (pour graph)**

On utilise ici un seul stream `s` :
```c
cudaStream_t s;
cudaStreamCreate(&s);
```
- **But :** tout sera capturé dans un graph, donc plus besoin de gérer plusieurs streams manuellement.

---

## 3. **Création et capture des CUDA Graphs**

On capture deux graphs, pour la parité (ping-pong des buffers). Tout le contenu d’une itération est capturé :

```c
cudaGraph_t       graph[2];
cudaGraphExec_t   graphExec[2];

for (int g=0; g<2; g++) {
    if (g==1) swap_ptrs(&d_a, &d_a_new);
    cudaStreamBeginCapture(s, cudaStreamCaptureModeGlobal);

    // Calcul bandes haut/bas (kernel sur s)
    dim3 grid_band((nx+BLOCK-1)/BLOCK, 1);
    jacobi_kernel_slice<<<grid_band, block, 0, s>>>(d_a, d_a_new, nx, 1, 2);
    jacobi_kernel_slice<<<grid_band, block, 0, s>>>(d_a, d_a_new, nx, local_ny, local_ny+1);

    // Calcul bulk intérieur (kernel sur s)
    if (iy_end_interior > iy_start_interior) {
        dim3 grid_interior((nx+BLOCK-1)/BLOCK, (iy_end_interior-iy_start_interior+BLOCK-1)/BLOCK);
        jacobi_kernel_slice<<<grid_interior, block, 0, s>>>(d_a, d_a_new, nx, iy_start_interior, iy_end_interior);
    }

    // NCCL halos dans le même stream/capture
    ncclGroupStart();
    ncclRecv(d_a_new + IDX(0,0,nx), nx, ncclDouble, prev, nccl_comm, s);
    ncclSend(d_a_new + IDX(1,0,nx), nx, ncclDouble, prev, nccl_comm, s);
    ncclRecv(d_a_new + IDX(ghost_ny-1,0,nx), nx, ncclDouble, next, nccl_comm, s);
    ncclSend(d_a_new + IDX(local_ny,0,nx), nx, ncclDouble, next, nccl_comm, s);
    ncclGroupEnd();

    cudaStreamEndCapture(s, &graph[g]);
    cudaGraphInstantiate(&graphExec[g], graph[g], NULL, NULL, 0);
    if (g==1) swap_ptrs(&d_a, &d_a_new);
}
```
- **`cudaStreamBeginCapture` / `cudaStreamEndCapture`** : tout ce qui est entre est enregistré dans un CUDA Graph.
- **`cudaGraphInstantiate`** : le graph capturé devient un objet exécutable.

---

## 4. **Lancement de la boucle Jacobi par graph**

Chaque itération lance simplement un des deux graphs, puis synchronise le stream :
```c
int idx = iter & 1;
cudaGraphLaunch(graphExec[idx], s);
cudaStreamSynchronize(s);
```
- **But :** coût d’appel quasi-nul, pipelines hardware, moins d’overhead CPU.

---

## 5. **Réduction de l’erreur toutes les N itérations**

Pour ne pas sortir du graph à chaque boucle, la norme de l’erreur (convergence) est calculée moins fréquemment :
```c
if (iter % 10 == 0) {
    error = thrust::transform_reduce(
        thrust::make_zip_iterator(thrust::make_tuple(ptr_new, ptr_old)),
        thrust::make_zip_iterator(thrust::make_tuple(ptr_new + local_N, ptr_old + local_N)),
        max_abs_diff(),
        0.0,
        thrust::maximum<double>());
    MPI_Allreduce(MPI_IN_PLACE, &error, 1, MPI_DOUBLE, MPI_MAX, MPI_COMM_WORLD);
}
```
- **But :** réduire l’overhead du critère d’arrêt, profiter du GPU au max.

---

## 6. **Nettoyage des graphs et objets CUDA**

On libère explicitement :
```c
cudaGraphExecDestroy(graphExec[0]);
cudaGraphExecDestroy(graphExec[1]);
cudaStreamDestroy(s);
ncclCommDestroy(nccl_comm);
```

---

## 7. **Intérêt**

- **Utilisation maximale du hardware** (pipelining, batch de kernels, latence minimale).
- **Scalabilité** sur gros clusters GPU/multi-nœuds (communiquer et calculer en masse).
- **Très peu d’overhead CPU** (le host n’intervient quasiment plus dans la boucle).

---

## 8. **Sur NCCL (dans ce contexte)**

- **NCCL dans un CUDA Graph** : permet de pipeline les transferts et calculs sur le device dans le même DAG hardware.
- **Tout reste sur le device**, sans synchronisation coûteuse côté host.

---

## Compilation et execution


In [1]:
%%bash
cd NCCL-graph

eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH

mpirun -np 4 ./jacobi_nccl 4096 4096 1000 1e-6 0


nvcc -O3 -arch=sm_90 -std=c++11 -ccbin=mpicxx -use_fast_math -I/home/scortinhal/.spack/userspace-installed/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/nccl-2.22.3-1-6mbbenokgc3egfcu6fgqxjwi7ea5oqc6/include -o jacobi_nccl jacobi.cu -L/home/scortinhal/.spack/userspace-installed/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/nccl-2.22.3-1-6mbbenokgc3egfcu6fgqxjwi7ea5oqc6/lib -lm -lmpi -lnccl -lcudart



Compilation terminated.
make: *** [Makefile:20: jacobi_nccl] Error 4


Process is interrupted.


# Partie 8 : NVSHMEM

NVSHMEM est une bibliothèque de communication destinée au calcul parallèle multi-GPU.
Son but est de faciliter et d’accélérer les échanges de données entre GPUs (sur le même nœud ou sur des nœuds différents) via des opérations de type mémoire partagée distribuée (Partitioned Global Address Space, PGAS).
Rôle principal

Partage de mémoire entre GPUs : chaque GPU (ou processus) a accès à un « heap symétrique » : une région de mémoire device accessible, en lecture/écriture, par tous les autres GPUs via des pointeurs d’adresse globale.
    
## 1. Initialisation de NVSHMEM

**Initialisation MPI et attribution GPU par processus local** :
```c
int local_rank;
MPI_Comm local_comm;
MPI_Comm_split_type(MPI_COMM_WORLD, MPI_COMM_TYPE_SHARED, rank, MPI_INFO_NULL, &local_comm);
MPI_Comm_rank(local_comm, &local_rank);
cudaSetDevice(local_rank);
cudaFree(0);
MPI_Comm_free(&local_comm);
```
- Permet d’avoir un GPU par processus MPI sur le nœud.

**Initialisation NVSHMEM avec MPI** :
```c
MPI_Comm mpi_comm = MPI_COMM_WORLD;
nvshmemx_init_attr_t attr;
attr.mpi_comm = &mpi_comm;
nvshmemx_init_attr(NVSHMEMX_INIT_WITH_MPI_COMM, &attr);
```
- NVSHMEM est initialisé à partir du communicateur MPI. Prend la main sur les échanges device-to-device.

---

## 2. Allocation de la mémoire sur le heap symétrique

```c
double *d_a     = (double*)nvshmem_malloc(bytes);
double *d_a_new = (double*)nvshmem_malloc(bytes);
```
- Permet à chaque processus de lire/écrire dans le segment mémoire de tous les autres PEs.

---

## 3. Communication des halos par NVSHMEM

Dans la boucle, on remplace MPI/NCCL par deux `nvshmem_double_put` (one-sided PUT) :

```c
// Envoi de la dernière ligne réelle dans le halo bas du voisin du bas
nvshmem_double_put(
    d_a + IDX(ghost_nys[next]-1, 0, nx),     // Destination : halo bas du voisin next
    d_a + IDX(local_ny, 0, nx),              // Source      : ma dernière ligne réelle
    nx, next);
// Envoi de la première ligne réelle dans le halo haut du voisin du haut
nvshmem_double_put(
    d_a + IDX(0, 0, nx),                     // Destination : halo haut du voisin prev
    d_a + IDX(1, 0, nx),                     // Source      : ma première ligne réelle
    nx, prev);
```
- L’adresse d’arrivée est calculée selon le découpage, grâce aux tableaux `ghost_nys`.
- Ces échanges se font **sans synchronisation immédiate** du récepteur.

**Synchronisation mémoire (fence) et barrière globale :**
```c
nvshmem_fence();
nvshmem_barrier_all();
```
- `nvshmem_fence()` : assure la visibilité de toutes les écritures PUT précédentes.
- `nvshmem_barrier_all()` : attend que tout le monde ait terminé les halos (prérequis à la cohérence du calcul).

---

## 4. Boucle Jacobi, calcul CUDA

Rien ne change côté kernel, à part l’utilisation du heap symétrique.

```c
jacobi_kernel<<<grid, block, 0, stream>>>(d_a, d_a_new, nx, ghost_ny);
cudaStreamSynchronize(stream);
```
- L’intégralité du calcul se fait sur la mémoire allouée NVSHMEM.

---

## 5. Réduction pour la convergence (MPI_Allreduce)

Le calcul de l’erreur max reste sur chaque GPU (via thrust), puis est globalisé :
```c
thrust::device_ptr<double> ptr_new(d_a_new);
thrust::device_ptr<double> ptr_old(d_a);
error = thrust::transform_reduce(
    thrust::make_zip_iterator(thrust::make_tuple(ptr_new, ptr_old)),
    thrust::make_zip_iterator(thrust::make_tuple(ptr_new + ghost_ny*nx, ptr_old + ghost_ny*nx)),
    max_abs_diff(),
    0.0,
    thrust::maximum<double>());
MPI_Allreduce(MPI_IN_PLACE, &error, 1, MPI_DOUBLE, MPI_MAX, MPI_COMM_WORLD);
```

---

## 6. Nettoyage NVSHMEM

À la fin, on libère la mémoire et on arrête NVSHMEM :
```c
if(d_a) nvshmem_free(d_a);
if(d_a_new) nvshmem_free(d_a_new);
cudaStreamDestroy(stream);
nvshmem_finalize();
MPI_Finalize();
```

---

## 7. Points clés sur NVSHMEM

- **PUT/GET** one-sided : Pas besoin que le voisin soit "prêt" (contrairement à MPI_Send/Recv).
- **Heap symétrique** : Obligatoire pour l'adressage direct de la mémoire entre processus.
- **Fence/barrière** : Pour garantir que les échanges sont terminés avant de lancer le calcul.
- **Remplace NCCL/MPI** pour les halos, mais la réduction d’erreur utilise toujours MPI_Allreduce (car NVSHMEM n'a pas d'opérations collectives avancées pour la réduction).
  
---


In [25]:
%%bash
cd NVSHMEM

# Charge NVSHMEM et NCCL
export NVSHMEM_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem
export LD_LIBRARY_PATH=$NVSHMEM_HOME/lib:$LD_LIBRARY_PATH
export CPATH=$NVSHMEM_HOME/include:$CPATH
export PATH=$NVSHMEM_HOME/bin:$PATH

eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH
export NVSHMEM_ENABLE_ALL_DEVICE_INLINING=1


make 
mpirun -np 4 ./jacobi_nvshmem 4096 4096 1000 1e-6 0


make: Nothing to be done for 'all'.
[NVSHMEM] Temps d'init avant NVSHMEM : 2.425963s
[NVSHMEM] Temps d'init NVSHMEM : 4.042471s
[NVSHMEM] Converged in 1000 iterations | Error: 2.42e-04
Temps de calcul Jacobi (seulement boucle)         : 0.133587s
Temps setup+calcul+affichage (après init SHMEM)   : 0.133753s
Temps total du programme (tout compris)           : 6.263870s


# Partie 9 : NVSHMEM-LTO

- Ajout du support de la compilation **LTO CUDA** (Link-Time Optimization) dans le Makefile :
    - Ajout du flag `-gencode=arch=compute_90,code=lto_90` à la variable NVCCFLAGS pour activer la génération du code intermédiaire LTO pour l’architecture sm_90.
    - Pas de modification du code source, uniquement une optimisation de la génération du binaire final via la chaîne de compilation NVCC pour de meilleures performances potentielles à l’exécution.
    - Optimiser les appels de fonctions entre différents fichiers sources CUDA (inlining, suppression de code mort, etc.)
    - Réorganiser ou fusionner des kernels s’il le juge pertinent.
    - Générer un code plus performant, parfois plus rapide à l’exécution que le même code compilé sans LTO.


In [27]:
%%bash
cd NVSHMEM-LTO


export NVSHMEM_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem
export CUDA_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.3
export LD_LIBRARY_PATH=$NVSHMEM_HOME/lib:$CUDA_HOME/lib64:$LD_LIBRARY_PATH
export CPATH=$NVSHMEM_HOME/include:$CUDA_HOME/include:$CPATH
export PATH=$NVSHMEM_HOME/bin:$CUDA_HOME/bin:$PATH

# Charge NCCL via Spack si tu utilises aussi NCCL ailleurs
eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH
export NVSHMEM_ENABLE_ALL_DEVICE_INLINING=1

make clean
make
#  lance avec mpirun (remplace nvshmrun si dispo) :
mpirun -np 4 ./jacobi_nvshmem 4096 4096 1000 1e-6 0


rm -f jacobi_nvshmem *.o *.qdrep *.sqlite
nvcc -O3 -arch=sm_90  -std=c++17 --expt-relaxed-constexpr -rdc=true -gencode=arch=compute_90,code=sm_90 -gencode=arch=compute_90,code=lto_90 -Xcompiler "-Wall -Wextra" -ccbin=mpicxx -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/include -I/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/include -o jacobi_nvshmem jacobi.cu -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/lib -lnvshmem -lnvshmem_host -L/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/lib64 -lcudart -lmpi
[NVSHMEM] Temps d'init avant NVSHMEM : 2.440521s
[NVSHMEM] Temps d'init NVSHMEM : 4.019447s
[NVSHMEM] Converged in 1000 iterations | Error: 2.42e-04
Temps de calcul Jacobi (seulement boucle)         : 0.135151s
Temps setup+calcul+affichage (après init SH

# Partie 10 : NVSHMEM-neighborhood_sync-lto


- Utilisation de **NVSHMEM** pour les échanges de halos entre GPUs (Remote Memory Access/RMA) :
    - Chaque processus écrit directement dans la mémoire du halo (bord) de ses voisins grâce à `nvshmem_double_put`.
    - Synchronisation mémoire assurée avec `nvshmem_fence()` puis barrière NVSHMEM pour garantir la cohérence avant le calcul.
- Gestion avancée de la **synchronisation neighborhood** sur device (entre GPU voisins) :
    - Ajout d’un kernel `syncneighborhood_kernel` pour notifier les voisins haut et bas que les halos sont prêts (utilisation de `nvshmemx_signal_op` et `nvshmem_uint64_wait_until_all` pour attendre les notifications des deux côtés).
- Initialisation :
    - Attribution automatique des GPU locaux par rang MPI intra-nœud.
    - Calcul dynamique des tailles de sous-domaines (ghost_ny) pour chaque processus, prise en compte des décompositions déséquilibrées.
    - Allocation de la mémoire sur le heap symétrique NVSHMEM (`nvshmem_malloc`) pour les tableaux et les variables de synchronisation, avec configuration dynamique de la variable d'environnement `NVSHMEM_SYMMETRIC_SIZE` pour garantir un espace mémoire suffisant.
    - Initialisation de NVSHMEM avec un communicateur MPI personnalisé via `nvshmemx_init_attr`.
- Calcul Jacobi toujours exécuté sur GPU via kernel CUDA, synchronisation des streams CUDA.
- Calcul de l’erreur max avec **Thrust** sur GPU (utilisation de la bibliothèque Thrust pour la réduction GPU des différences a_new/a_old), puis MPI_Allreduce pour la convergence globale.
- Mesure détaillée des temps : temps avant et après initialisation NVSHMEM, temps de calcul pur, temps total global, etc.
- Rassemblement final et affichage de la grille sur le rang 0 avec gestion correcte des tailles locales, comme dans les versions précédentes.


In [29]:
%%bash
cd NVSHMEM-neighborhood_sync-lto

# Charge les bonnes librairies NVSHMEM et CUDA
export NVSHMEM_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem
export CUDA_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.3
export LD_LIBRARY_PATH=$NVSHMEM_HOME/lib:$CUDA_HOME/lib64:$LD_LIBRARY_PATH
export CPATH=$NVSHMEM_HOME/include:$CUDA_HOME/include:$CPATH
export PATH=$NVSHMEM_HOME/bin:$CUDA_HOME/bin:$PATH

# Charge NCCL via Spack si nécessaire
eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH

# Active l'inlining maximal NVSHMEM
export NVSHMEM_ENABLE_ALL_DEVICE_INLINING=1

make clean && make

mpirun -np 4 ./jacobi_nvshmem 4096 4096 1000 1e-6 0




rm -f jacobi_nvshmem *.o *.qdrep *.sqlite
nvcc -O3 -arch=sm_90  -std=c++17 --expt-relaxed-constexpr -rdc=true -gencode=arch=compute_90,code=sm_90 -gencode=arch=compute_90,code=lto_90 -Xcompiler "-Wall -Wextra" -ccbin=mpicxx -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/include -I/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/include -o jacobi_nvshmem jacobi.cu -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/lib -lnvshmem -lnvshmem_host -L/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/lib64 -lcudart -lmpi
[NVSHMEM-neighSync] Converged in 1000 iterations | Error: 2.42e-04
Temps de calcul Jacobi (seulement boucle)         : 0.129805s
Temps setup+calcul+affichage (après init SHMEM)   : 0.130036s
Temps d'init avant NVSHMEM                        : 2.443952s
T

# Partie 11 : nvshmem-norm_overlap-neighborhood_sync+lto 

- le calcul sur la grille locale est vraiment overlap avec l'échange des halos grâce à un découpage fin des kernels et à la synchronisation asynchrone device-side.

- On découpe la mise à jour Jacobi en :
    - `jacobi_inner_kernel` (zone intérieure, ne dépend pas du halo) qui est lancé pendant que les halos sont envoyés aux voisins.
    - Une fois l'échange de halos terminé (vérifié via le kernel `syncneighborhood_kernel` qui utilise des signaux device NVSHMEMX),  
      on lance `jacobi_border_kernel` sur les deux bandes frontalières haut/bas qui ont besoin du halo à jour.
  Ce découpage permet de **masquer le coût de communication** et d'accélérer la convergence pour des grosses grilles.

- **Synchronisation neighborhood**: au lieu d'une barrière globale (`nvshmem_barrier_all`), chaque processus attend juste un signal de ses deux voisins directs (haut/bas) pour la cohérence des halos : cela limite l'attente et améliore l'overlap comm/calcul.

- Le reste de la structure générale (allocation heap symétrique, gestion des tailles fantômes variables, calcul erreur avec thrust, réduction MPI_Allreduce, réassemblage final) reste identique à la version précédente.

- L'objectif est de **maximiser le recouvrement entre calcul local et communication** des halos, ce qui est particulièrement efficace sur des architectures GPU modernes.



In [31]:
%%bash
cd NVSHMEM-norm_overlap-neighborhood_sync+lto

# Charge les bonnes librairies NVSHMEM et CUDA
export NVSHMEM_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem
export CUDA_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.3
export LD_LIBRARY_PATH=$NVSHMEM_HOME/lib:$CUDA_HOME/lib64:$LD_LIBRARY_PATH
export CPATH=$NVSHMEM_HOME/include:$CUDA_HOME/include:$CPATH
export PATH=$NVSHMEM_HOME/bin:$CUDA_HOME/bin:$PATH

# Charge NCCL via Spack si nécessair

eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH

# Active l'inlining maximal NVSHMEM
export NVSHMEM_ENABLE_ALL_DEVICE_INLINING=1

make clean && make

mpirun -np 4 ./jacobi_nvshmem 4096 4096 1000 1e-6 0




rm -f jacobi_nvshmem *.o *.qdrep *.sqlite
nvcc -O3 -arch=sm_90  -std=c++17 --expt-relaxed-constexpr -rdc=true -gencode=arch=compute_90,code=sm_90 -gencode=arch=compute_90,code=lto_90 -Xcompiler "-Wall -Wextra" -ccbin=mpicxx -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/include -I/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/include -o jacobi_nvshmem jacobi.cu -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/lib -lnvshmem -lnvshmem_host -L/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/lib64 -lcudart -lmpi
[NVSHMEM-norm_overlap-neighSync+LTO] Converged in 1000 iterations | Error: 2.42e-04
Temps de calcul Jacobi (seulement boucle)         : 0.114876s
Temps setup+calcul+affichage (après init SHMEM)   : 0.115086s
Temps d'init avant NVSHMEM                    

# Partie 12 : nvshmem-norm_overlap-neighborhood_sync+lto[1]  


- **Ici, c'est une version "baseline" (de base, séquentielle côté device)** :
    - On utilise **un seul kernel Jacobi** (`jacobi_kernel_full`) qui fait tout le calcul d'un coup sur tout le sous-domaine local, sans découpage ni overlap calcul/communication.
    - **Pas de découpage** en "zone intérieure" et "zone de bord" : tout est fait d’un bloc, donc la communication des halos ne peut pas être masquée par le calcul local.
    - **Pas de synchronisation neighborhood device** (pas de kernel de synchronisation avec les voisins via NVSHMEMX), ni de stratégie fine pour attendre juste les voisins.
    - On se contente d’un schéma classique : 
        - On fait tout le calcul local,
        - Puis l’erreur est calculée (avec thrust),
        - Puis un `MPI_Allreduce` pour la convergence globale,
        - Et on passe à l’itération suivante.

- **En résumé :**  
  - Version "simple" qui sert de référence pour les perfs : pas d’overlap calcul/comm, pas d’optimisation, pas de synchronisation fine, **juste un Jacobi standard sur tout le domaine local**.
  - Cela permet de comparer l’apport réel des optimisations dans les autres versions (découpage kernels + overlap, synchronisation neighborhood, etc).


In [35]:
%%bash
cd NVSHMEM-norm_overlap-neighborhood_sync+lto[1]

# Charge les bonnes librairies NVSHMEM et CUDA
export NVSHMEM_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem
export CUDA_HOME=/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/cuda/12.3
export LD_LIBRARY_PATH=$NVSHMEM_HOME/lib:$CUDA_HOME/lib64:$LD_LIBRARY_PATH
export CPATH=$NVSHMEM_HOME/include:$CUDA_HOME/include:$CPATH
export PATH=$NVSHMEM_HOME/bin:$CUDA_HOME/bin:$PATH

# Charge NCCL via Spack si nécessair

eval $(spack load --sh nccl)
export LD_LIBRARY_PATH=$(spack location -i nccl)/lib:$LD_LIBRARY_PATH

# Active l'inlining maximal NVSHMEM
export NVSHMEM_ENABLE_ALL_DEVICE_INLINING=1

make clean && make

mpirun -np 4 ./jacobi_nvshmem 4096 4096 1000 1e-6 0




rm -f jacobi_nvshmem *.o *.qdrep *.sqlite
nvcc -O3 -arch=sm_90  -std=c++17 --expt-relaxed-constexpr -rdc=true -gencode=arch=compute_90,code=sm_90 -gencode=arch=compute_90,code=lto_90 -Xcompiler "-Wall -Wextra" -ccbin=mpicxx -I/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/include -I/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/include -o jacobi_nvshmem jacobi.cu -L/apps/2025/manual_install/nvhpc/24.11/Linux_aarch64/24.11/comm_libs/12.6/nvshmem/lib -lnvshmem -lnvshmem_host -L/apps/2025/spack_install/linux-rhel9-neoverse_v2/linux-rhel9-neoverse_v2/gcc-11.4.1/cuda-12.6.2-3mzltpzgs4dekx5xvbqzz2no3j3tkcxq/lib64 -lcudart -lmpi
[NVSHMEM-baseline-single-rank] Converged in 1000 iterations | Error: 2.42e-04
Temps de calcul Jacobi (seulement boucle)         : 0.092085s
Temps setup+calcul+affichage (après init SHMEM)   : 0.092252s
Temps d'init avant NVSHMEM                        : 

![MPI-GPU](Graph.PNG)

![MPI-GPU](Result.PNG)